# 🧪 ADK 與 A2A 應用程式測試

這個 notebook 示範如何測試實作 Agent2Agent (A2A) 協定的 ADK (Agent Development Kit) 應用程式。

**涵蓋範圍：**
- 本地端 (local) 與遠端 (remote) 測試
- Agent Engine 和 Cloud Run 兩種部署方式的測試方法

> **重點說明：** A2A 協定是 Agent 之間互相溝通的標準化協定，讓不同的 Agent 可以相互呼叫和協作。

## 設定您的環境

> **注意：** 為了獲得最佳結果，請使用與 `uv` 本地開發時相同的 `.venv` 虛擬環境，以確保相依套件的相容性並避免環境相關的問題。
> 
> **重點說明：** 使用相同的虛擬環境可以避免套件版本衝突，確保開發環境與測試環境一致。

In [ ]:
# 如果您沒有使用 uv 建立的虛擬環境，請取消以下幾行的註解
# 重點：這會將父目錄加入 Python 路徑，並安裝必要的套件
# import sys

# sys.path.append("../")
# !pip install google-cloud-aiplatform a2a-sdk --upgrade

### 匯入函式庫

In [ ]:
# ruff: noqa
# 匯入所有必要的函式庫
import asyncio  # 非同步處理
import json  # JSON 資料處理
import os  # 作業系統相關功能
import requests  # HTTP 請求
import uuid  # 唯一識別碼生成

import vertexai  # Vertex AI 客戶端
# A2A SDK 相關型別定義
from a2a.types import (
    Message,
    MessageSendParams,
    Part,
    Role,
    SendStreamingMessageRequest,
    TextPart,
)
from IPython.display import Markdown, display  # Jupyter 顯示工具
# ADK 核心元件
from google.adk.artifacts import InMemoryArtifactService  # 記憶體內的 Artifact 服務
from google.adk.sessions import InMemorySessionService  # 記憶體內的 Session 服務

from app.agent_engine_app import AgentEngineApp  # Agent Engine 應用程式
# 測試輔助函式
from tests.helpers import (
    build_get_request,
    build_post_request,
    poll_task_completion,
)

### 初始化 Vertex AI 客戶端

> **重點說明：** 這裡設定 Vertex AI 的連線參數，`location` 決定了服務運行的區域。

In [ ]:
# 初始化 Vertex AI 客戶端
# 重點：設定 GCP 的服務區域為美國中部
LOCATION = "us-central1"

client = vertexai.Client(
    location=LOCATION,
)

## 如果您使用的是 Agent Engine

更多文件請參考：[Agent Engine 概述](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

> **重點說明：** Agent Engine 是 Google Cloud 提供的託管 Agent 執行環境，可以簡化部署和管理流程。

### 遠端測試

> **重點說明：** 遠端測試是針對已部署到 Agent Engine 的 Agent 進行測試，透過 API 進行呼叫。

In [ ]:
# 設定為 None 以從 ./deployment_metadata.json 自動偵測，或手動指定
# 格式："projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
# 重點：這個 ID 唯一識別已部署的 Reasoning Engine
REASONING_ENGINE_ID = None

if REASONING_ENGINE_ID is None:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass

print(f"使用 REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")

# 從 REASONING_ENGINE_ID 中提取 project_id、location 和 engine_id
# 重點：解析完整的資源路徑以取得各個元件
parts = REASONING_ENGINE_ID.split("/")
project_id = parts[1]
location = parts[3]
engine_id = parts[5]

# 建構 API 端點
# 重點：組合成完整的 REST API 呼叫路徑
base_url = f"https://{location}-aiplatform.googleapis.com"
a2a_base_path = f"/v1beta1/projects/{project_id}/locations/{location}/reasoningEngines/{engine_id}/a2a/v1"

print(f"Base URL: {base_url}")
print(f"A2A base path: {a2a_base_path}")

#### 取得 Agent Card

> **重點說明：** Agent Card 包含 Agent 的基本資訊、能力和支援的技能，是 A2A 協定的核心元資料。

In [ ]:
# 使用 REST API 取得 Agent Card
# 重點：需要先取得 Google Cloud 的驗證令牌
import google.auth
import google.auth.transport.requests

# 取得驗證令牌
creds, project = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

# 設定 HTTP 標頭，包含驗證資訊
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {creds.token}"}

# 發送 GET 請求以取得 Agent Card
# 重點：/card 端點會返回 Agent 的詳細資訊
response = requests.get(
    f"{base_url}{a2a_base_path}/card",
    headers=headers,
)

print(f"回應狀態碼: {response.status_code}")

if response.status_code == 200:
    remote_a2a_agent_card = response.json()
    print(f"Agent 名稱: {remote_a2a_agent_card.get('name')}")
    print(f"URL: {remote_a2a_agent_card.get('url')}")
    print(
        f"技能清單: {[s.get('description') for s in remote_a2a_agent_card.get('skills', [])]}"
    )
    print(f"協定版本: {remote_a2a_agent_card.get('protocolVersion')}")
else:
    print(f"錯誤: {response.text}")

#### 傳送訊息

> **重點說明：** 這裡示範如何使用 A2A 協定發送訊息給 Agent，並取得任務 ID 用於後續追蹤。

In [ ]:
# 使用 A2A REST API 發送訊息
import google.auth
import google.auth.transport.requests

# 取得驗證令牌
creds, project = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {creds.token}"}

# 準備訊息資料
# 重點：messageId 必須是唯一的，用於追蹤和除錯
data = {
    "message": {
        "messageId": f"msg-{os.urandom(8).hex()}",
        "content": [{"text": "What is the weather in New York?"}],
        "role": "ROLE_USER",
    }
}

# 發送 POST 請求到 message:send 端點
# 重點：這是非同步呼叫，會立即返回任務 ID
response = requests.post(
    f"{base_url}{a2a_base_path}/message:send",
    headers=headers,
    json=data,
)

print(f"回應狀態碼: {response.status_code}")

if response.status_code == 200:
    response_data = response.json()
    task_id = response_data["task"]["id"]
    print(f"任務已啟動，ID: {task_id}")
else:
    print(f"錯誤: {response.text}")

#### 輪詢回應結果

> **重點說明：** Agent 處理是非同步的，需要定期輪詢任務狀態直到完成或失敗。

In [ ]:
# 使用 REST API 輪詢任務完成狀態
# 重點：設定最大嘗試次數避免無限等待
max_attempts = 30
for attempt in range(max_attempts):
    poll_response = requests.get(
        f"{base_url}{a2a_base_path}/tasks/{task_id}",
        headers=headers,
    )

    if poll_response.status_code != 200:
        print(f"輪詢失敗，狀態碼: {poll_response.status_code}")
        break

    result = poll_response.json()
    task_state = result.get("status", {}).get("state")
    print(f"嘗試 {attempt + 1}: {task_state}")

    # 檢查任務是否完成
    if task_state == "TASK_STATE_COMPLETED":
        print("任務完成！")
        break
    elif task_state in ["TASK_STATE_FAILED", "TASK_STATE_CANCELLED"]:
        print(f"任務失敗: {result}")
        break

    # 等待 1 秒後再次檢查
    await asyncio.sleep(1)

# 提取並顯示結果 artifacts
# 重點：artifacts 包含 Agent 產生的所有輸出內容
if "artifacts" in result and result["artifacts"]:
    for artifact in result["artifacts"]:
        if artifact.get("parts"):
            for part in artifact["parts"]:
                if "text" in part:
                    display(Markdown(f"**回答**:\n {part['text']}"))
                else:
                    print("無法從 artifact parts 中提取文字內容。")
else:
    print("結果中沒有找到 artifacts")

### 本地端測試

您可以直接在您的環境中匯入 AgentEngineApp 類別。

**在本地端執行 Agent 的步驟：**

1. 確保所有必要的套件都已安裝在您的環境中
2. 建議使用 `uv` 工具建立的虛擬環境
3. 您可以從 Agent 的根目錄執行 `make install` 來設定此環境
4. 然後在 Jupyter notebook 中選擇此 kernel (專案中的 .venv 資料夾)，以確保所有相依套件都可用

> **重點說明：** 本地端測試不需要部署到雲端，適合快速開發和除錯。

In [ ]:
# 匯入並設定 Agent Engine
# 重點：set_up() 方法會初始化 Agent 的所有必要元件
from app.agent_engine_app import agent_engine

agent_engine.set_up()

#### 驗證自訂方法是否已註冊

> **重點說明：** 檢查 Agent Engine 是否正確註冊了所有自訂的操作方法。

In [ ]:
# 測試註冊的操作
# 重點：這會返回所有已註冊的操作清單
test = agent_engine.register_operations()
print(test)

#### 取得 Agent Card

In [ ]:
# 建立 GET 請求並取得 Agent Card
# 重點：本地端測試不需要驗證令牌
request = build_get_request(None)
response = await agent_engine.handle_authenticated_agent_card(
    request=request, context=None
)
print(response)

#### 傳送訊息

In [ ]:
# 準備訊息資料
# 重點：建立唯一的訊息 ID 用於追蹤
message_data = {
    "message": {
        "messageId": f"msg-{os.urandom(8).hex()}",
        "content": [{"text": "What is the weather in New York?"}],
        "role": "ROLE_USER",
    },
}

# 建立 POST 請求
request = build_post_request(message_data)

# 呼叫 Agent Engine 的訊息處理方法
# 重點：這是非同步呼叫，使用 await
response = await agent_engine.on_message_send(request=request, context=None)
print(response)

#### 輪詢回應結果

In [ ]:
# 從回應中取得任務 ID
task_id = response["task"]["id"]
print(f"任務 ID 為: {task_id}")

# 使用輔助函式輪詢完成狀態
# 重點：poll_task_completion 會自動處理輪詢邏輯
final_response = await poll_task_completion(agent_engine, task_id)

# 提取並顯示 artifacts
for artifact in final_response["artifacts"]:
    if artifact["parts"] and "text" in artifact["parts"][0]:
        display(Markdown(f"**回答**:\n {artifact['parts'][0]['text']}"))
    else:
        print("無法從 artifact parts 中提取文字內容。")

#### 註冊回饋

> **重點說明：** 可以記錄使用者對 Agent 回應的評分和回饋，用於後續改進。

In [ ]:
# 註冊使用者回饋
# 重點：包含評分、文字回饋、使用者 ID 和 session ID
agent_engine.register_feedback(
    feedback={
        "score": 5,  # 評分：1-5
        "text": "Great response!",  # 回饋內容
        "user_id": "test-user-123",  # 使用者識別碼
        "session_id": "test-session-123",  # 會話識別碼
    }
)

## 如果您使用的是 Cloud Run

> **重點說明：** Cloud Run 是 Google Cloud 的無伺服器容器平台，適合部署 HTTP 服務。

### 遠端測試

更多關於向 Cloud Run 服務驗證 HTTPS 請求的資訊，請參考：
[Cloud Run 驗證文件](https://cloud.google.com/run/docs/triggering/https-request)

**遠端測試的特點：**

- 使用已部署的服務 URL，而非 localhost
- 使用 GCP identity tokens 進行驗證，而非本地憑證

> **重點說明：** 遠端測試模擬真實的生產環境，可以驗證部署後的實際行為。

In [ ]:
# 取得 Google Cloud 身份令牌
# 重點：-q 參數表示安靜模式，只輸出令牌
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
# 請將此處替換為您的 Cloud Run 服務 URL
# 重點：可以從 Cloud Run 控制台取得完整的服務 URL
SERVICE_URL = "YOUR_SERVICE_URL_HERE"

使用 A2A 協定發送訊息

> **重點說明：** 這裡示範使用串流 (streaming) 方式接收 Agent 回應，可以即時取得部分結果。

In [ ]:
# 建立 A2A 訊息請求
# 重點：使用 A2A SDK 的型別定義來建構訊息
message = Message(
    message_id=f"msg-user-{uuid.uuid4()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="Hello! Weather in New York?"))],
)

request = SendStreamingMessageRequest(
    id=f"req-{uuid.uuid4()}",
    params=MessageSendParams(message=message),
)

# 設定包含驗證資訊的標頭
# 重點：使用 Bearer token 進行身份驗證
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

# 發送串流請求到 A2A 端點
# 重點：stream=True 啟用串流模式，timeout 設定請求超時時間
response = requests.post(
    f"{SERVICE_URL}/a2a/app",
    headers=headers,
    json=request.model_dump(mode="json", exclude_none=True),
    stream=True,
    timeout=60,
)

print(f"回應狀態碼: {response.status_code}")

# 解析串流 A2A 回應
# 重點：串流回應以 "data: " 開頭的 Server-Sent Events 格式傳送
for line in response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]  # 移除 "data: " 前綴
            event = json.loads(event_json)
            print(f"收到事件: {event}")

### 本地端測試

> 您可以透過 `make local-backend` 命令在本地端執行應用程式。

**功能說明：** 使用 A2A 協定向本地端後端服務發送訊息，並接收串流回應。

> **重點說明：** 本地端測試不需要身份驗證，適合快速開發和除錯。

In [ ]:
# 建立 A2A 訊息請求
message = Message(
    message_id=f"msg-user-{uuid.uuid4()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="Hello! Weather in New York?"))],
)

request = SendStreamingMessageRequest(
    id=f"req-{uuid.uuid4()}",
    params=MessageSendParams(message=message),
)

# 設定標頭（本地端不需要驗證）
headers = {"Content-Type": "application/json"}

# 發送串流請求到本地端 A2A 端點
# 重點：使用 localhost 連接本地服務，預設埠號為 8000
response = requests.post(
    "http://127.0.0.1:8000/a2a/app",
    headers=headers,
    json=request.model_dump(mode="json", exclude_none=True),
    stream=True,
    timeout=60,
)

print(f"回應狀態碼: {response.status_code}")

# 解析串流 A2A 回應
for line in response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"收到事件: {event}")